 # Pridobivanje globalnih podnebnih podatkov

## 1. Uvod


 ## 2. Nastavitev okolja
 ### 2.1 Uvoz potrebnih knjižnic

-  Pri pogonu ( `Run All` ) se avtomatsko naložijo vse potrebne knjižnice.

-  V primeru napake, je treba prvo pognati funkcijo `zahteve_namestitev.py`, potem pa v tem notebooku `Restart Kernel` in ponovno `Run All`

In [16]:
from zahteve_namestitev import instalacija
instalacija()

import matplotlib.pyplot as plt         # Grafi
from datetime import datetime           # Datumi
import pandas as pd                     # Pandas
import numpy as np                      # Numerično
from scipy import stats                 # Statistika
from tqdm import tqdm                   # Progress bar
from pathlib import Path                # Poti do datotek
import glob                             # Iskanje datotek
import warnings                         # Prikazovanje težav
import os                               # Operacijski sistem
import seaborn as sns                   # Statistične vizualizacije
from meteostat import Point, Daily      # Geografska lokacija in dnevni podatki o temperaturi

🔧 Nameščam 1 manjkajočih paketov...
✅ Paketi uspešno nameščeni
🔄 Ponovno zaganjam kernel...


<IPython.core.display.Javascript object>

 ### 2.2 Nastavimo poti in ustvarimo mape
-  Nastavimo osnovne poti do map v uporabi,

-  `.mkdir()` - Preveri obstoj potrebnih map in jih v primeru, da še ne obstajajo naredi;

-  Nastavimo spremenljivki za začetni in končni datum,

-  Preverimo lokacijo vseh map.

In [17]:
Glavna_mapa = Path.cwd()
Podatkovna_baza = Glavna_mapa / "Podatki" / "nepredelani"
Rezultati = Glavna_mapa / "Podatki" / "rezultati"

Podatkovna_baza.mkdir(parents=True, exist_ok=True)
Rezultati.mkdir(parents=True, exist_ok=True)

zacetek = datetime(1950, 1, 1)
konec = datetime(2023, 12, 31)

print(f"""
Glavni imenik:      {Glavna_mapa}
Mapa za podatke:    {Podatkovna_baza}
Mapa za rezultate:  {Rezultati} """)


Glavni imenik:      c:\Users\jan\Projektna_naloga_UVP
Mapa za podatke:    c:\Users\jan\Projektna_naloga_UVP\Podatki\nepredelani
Mapa za rezultate:  c:\Users\jan\Projektna_naloga_UVP\Podatki\rezultati 


 ### 2.3 Pomožne skripte
-  `ustvari_mapo()` - ustvari novo mapo na podani poti;

-  `shrani_podatke()` - datoteko v formatu DataFrame pretvori v CSV in shrani pod imenom: 
   
   `[ime]_[leto]_temperature.csv`

In [18]:
def ustvari_mapo(pot):

    os.makedirs(pot, exist_ok=True)

def shrani_podatke(df, lokacija_ime, leto):

    pot = Podatkovna_baza / lokacija_ime
    ustvari_mapo(pot)
    datoteka = pot / f"{lokacija_ime}_{leto}_temperature.csv"
    df.to_csv(datoteka, index=False)

 ### 2.4 Lokacije
-  Definiramo izbrane lokacije v slovarju,

-  Poleg imena jim dodamo njihovo geografsko lokacijo in `klimatski_pas`.

   ( latitude / `lat` = širina,  longitude / `lon` = dolžina )"

In [19]:
lokacije = [
    # Severna vzhodna polobla (Evropa, Azija)
    {'ime': 'Tokyo', 'lat': 35.68, 'lon': 139.77},       # Azija - zmerno podnebje (vaša izbira)
    {'ime': 'Berlin', 'lat': 52.52, 'lon': 13.41},       # Evropa - kontinentalno (vaša izbira)
    {'ime': 'Reykjavik', 'lat': 64.13, 'lon': -21.82},   # Subarktično (premaknjeno iz severne zahodne)
    {'ime': 'Prag', 'lat': 50.09, 'lon': 14.42},        # 98% (zmerno)
    # Severna zahodna polobla (Severna Amerika)
    {'ime': 'Alert', 'lat': 82.50, 'lon': -62.33},     # Arktična podnebna cona (najsevernejša stalna postaja na svetu)
    {'ime': 'Denver', 'lat': 39.74, 'lon': -104.99},     # Višinsko podnebje (vaša izbira)
    {'ime': 'Toronto', 'lat': 43.70, 'lon': -79.42},     # 88% → 92% (kontinentalno)
    # Južna vzhodna polobla (Avstralija, Antarktika)
    {'ime': 'Sydney', 'lat': -33.87, 'lon': 151.21},     # Subtropsko (vaša izbira)
    {'ime': 'McMurdo_Station', 'lat': -77.85, 'lon': 166.67}, # Polarno (vaša izbira)
    {'ime': 'Auckland', 'lat': -36.85, 'lon': 174.76}, # 89% → 93% (morsko)
    # Južna zahodna polobla (Južna Amerika)
    {'ime': 'Santiago', 'lat': -33.45, 'lon': -70.67},   # Sredozemsko (vaša izbira)
    {'ime': 'Lima', 'lat': -12.05, 'lon': -77.03},      # 70% → ~82% (interpolirano, puščavsko)
]


 ## 3 Pridobivanje podatkov
-  Definiramo funkcijo `pridobi_podatke_za_lokacije`, ki pridobi podatke za vse
   definirane lokacije v določenem časovnem obdobju.

-  Zbiranje podatkov za vsako lokacijo znotraj zanke `for` se kaže preko progress bara (paket `tqdm`).

-  V zanki `Try`, ki zajema morebitne napake, prvo nastavimo spremenljivki:
    
    -  Spremenljivko `tocka` na geografsko lokacijo kraja s funkcijo `Point()`
       in spremenljivko `podatki` na vse dostopne dnevne informacije o vremenu s funkcijo `Daily()`.

    -  Funkcija `fetch()` nato pošlje zahtevo k Meteostat API in podatke dobi v obliki `df`
       (DataFrame).

    -  DataFrame-u spremenimo strukturo in znotraj zanke `for` s funkcijo `shrani_podatke()`
    vanj shranimo podatke po dnevih za eno leto 

-  Klic funkcije `pridobi_podatke_za_lokacije()` shrani rezultate pridobivanja podatkov v
   spremenljivki `uspeh` in `neuspeh`

In [ ]:
import time
def pridobi_podatke_za_lokacije(lokacije, zacetek, konec):

    uspeh = 0
    neuspeh = 0

    with tqdm(lokacije, 
              desc=" Pridobivanje podatkov :",
              bar_format="{l_bar}{bar:30}[{remaining}]"
              ) as pbar:
        for lokacija in pbar:
            pbar.set_description(f"🔍 {lokacija['ime'][:15]:<15}") 
            
            try:
                tocka = Point(lokacija['lat'], lokacija['lon'])

                pbar.set_description(f" {lokacija['ime'].ljust(15)}")

                podatki = Daily(tocka, zacetek, konec)
                df = podatki.fetch()

                if df is None or df.empty:
                    neuspeh += 1   #Preveri DataFrame
                    pbar.set_postfix_str(" Ni podatkov", refresh=False)
                    continue
                
                df = df.reset_index()
                if 'tavg' in df.columns:
                    df['temperatura_avg'] = df['tavg']
                else:
                    df['temperatura_avg'] = None

                df = df[['time', 'temperatura_avg']].copy()

                leta = df['time'].dt.year.unique()
                with tqdm(leta, desc=f"Shranjujem {lokacija['ime']}", leave=False) as year_pbar:
                    for leto in year_pbar:
                        df_leto = df[df['time'].dt.year == leto].copy()
                        if not df_leto['temperatura_avg'].isnull().all():
                            shrani_podatke(df_leto, lokacija['ime'], leto)
                
                uspeh += 1

            except Exception as e:
                # Bolj specifičen error handling
                if "No data available" in str(e):
                    neuspeh += 1
                    continue
                print(f"Napaka pri {lokacija['ime']}: {str(e)}")
                neuspeh += 1
    
        return uspeh, neuspeh

uspeh, neuspeh = pridobi_podatke_za_lokacije(lokacije, zacetek, konec)

print(f"""
Uspešno:  {uspeh} lokacij
Neuspešno: {neuspeh} lokacij
Mapa s podatki: {Podatkovna_baza} """)


 Lima           : 100%|██████████████████████████████[00:00]    


Uspešno:  12 lokacij
Neuspešno: 0 lokacij
Mapa za podatke: c:\Users\jan\Projektna_naloga_UVP\Podatki\nepredelani 
